<a href="https://colab.research.google.com/github/elenagood2014-maker/lips/blob/main/wave_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# ПОЛНЫЙ КОД WAV2LIP ДЛЯ GOOGLE COLAB - СКОПИРУЙ ВЕСЬ КОД В ОДНУ ЯЧЕЙКУ
# ==============================================================================

import os
import shutil
import subprocess
import tempfile
from threading import Thread
import time
import torch
import urllib.request

# ==============================================================================
# ШАГ 1: ПОДКЛЮЧЕНИЕ GOOGLE DRIVE
# ==============================================================================
print("="*70)
print("ШАГ 1: ПОДКЛЮЧЕНИЕ GOOGLE DRIVE")
print("="*70)

from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive подключён!\n")

# ==============================================================================
# ШАГ 2: УСТАНОВКА ЗАВИСИМОСТЕЙ
# ==============================================================================
print("="*70)
print("ШАГ 2: УСТАНОВКА ЗАВИСИМОСТЕЙ")
print("="*70)

os.system('pip install -q flask-ngrok pyngrok 2>&1 | grep -v "already satisfied"')
os.system('apt-get -qq install ffmpeg')
print("✅ Зависимости установлены\n")

# ==============================================================================
# ШАГ 3: КЛОНИРОВАНИЕ WAV2LIP
# ==============================================================================
print("="*70)
print("ШАГ 3: КЛОНИРОВАНИЕ WAV2LIP")
print("="*70)

if not os.path.exists('Wav2Lip'):
    os.system('git clone https://github.com/Rudrabha/Wav2Lip.git')
    print("✅ Wav2Lip клонирован")
else:
    print("✅ Wav2Lip уже существует")

os.chdir('/content/Wav2Lip')
print(f"📁 Текущая директория: {os.getcwd()}\n")

# ==============================================================================
# ШАГ 4: УСТАНОВКА REQUIREMENTS
# ==============================================================================
print("="*70)
print("ШАГ 4: УСТАНОВКА REQUIREMENTS")
print("="*70)

os.system('pip install -q -r requirements.txt 2>&1 | grep -v "already satisfied"')
os.system('pip install -q flask requests pillow 2>&1 | grep -v "already satisfied"')
print("✅ Requirements установлены\n")

# ==============================================================================
# ШАГ 5: КОПИРОВАНИЕ ВАШЕЙ МОДЕЛИ ИЗ GOOGLE DRIVE
# ==============================================================================
print("="*70)
print("ШАГ 5: КОПИРОВАНИЕ МОДЕЛИ")
print("="*70)

# ⚠️  ИЗМЕНИ ЭТОТ ПУТЬ НА СВОЙ!
SOURCE_MODEL = "/content/drive/MyDrive/Wav2Lip-SD-GAN.pt"

os.makedirs('checkpoints', exist_ok=True)
TARGET_MODEL = "checkpoints/Wav2Lip-SD-GAN.pt"

if os.path.exists(SOURCE_MODEL):
    print(f"📦 Копируем модель из: {SOURCE_MODEL}")
    shutil.copy(SOURCE_MODEL, TARGET_MODEL)
    size = os.path.getsize(TARGET_MODEL) / (1024*1024)
    print(f"✅ Модель скопирована! Размер: {size:.2f} MB")
    print(f"📁 Путь к модели: {TARGET_MODEL}\n")
else:
    print(f"❌ ОШИБКА: Файл не найден: {SOURCE_MODEL}")
    print("\n🔍 Содержимое вашего Google Drive:")
    os.system('ls -lah /content/drive/MyDrive/ | head -20')
    print("\n⚠️  ИСПРАВЬ ПУТЬ SOURCE_MODEL в коде!\n")
    raise FileNotFoundError(f"Модель не найдена: {SOURCE_MODEL}")

# ==============================================================================
# ШАГ 6: СКАЧИВАНИЕ FACE DETECTION МОДЕЛИ
# ==============================================================================
print("="*70)
print("ШАГ 6: СКАЧИВАНИЕ FACE DETECTION МОДЕЛИ")
print("="*70)

FACE_DET_PATH = "face_detection/detection/sfd/s3fd.pth"
os.makedirs('face_detection/detection/sfd', exist_ok=True)

if not os.path.exists(FACE_DET_PATH):
    print("📥 Скачиваем face detection модель...")
    try:
        urllib.request.urlretrieve(
            "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth",
            FACE_DET_PATH
        )
        print("✅ Face detection модель скачана\n")
    except Exception as e:
        print(f"⚠️  Ошибка: {e}")
        print("Попробуйте скачать вручную с:")
        print("https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth\n")
else:
    print("✅ Face detection модель уже есть\n")

# ==============================================================================
# ШАГ 7: ЗАГРУЗКА МОДЕЛИ WAV2LIP
# ==============================================================================
print("="*70)
print("ШАГ 7: ЗАГРУЗКА МОДЕЛИ WAV2LIP")
print("="*70)

from models import Wav2Lip

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️  Устройство: {device}")

MODEL_PATH = TARGET_MODEL
print(f"📦 Размер модели: {os.path.getsize(MODEL_PATH) / (1024*1024):.2f} MB")
print(f"🔄 Загружаем модель...\n")

try:
    checkpoint = torch.load(MODEL_PATH, map_location=device, weights_only=False)
    print(f"✅ Файл загружен! Тип: {type(checkpoint)}")

    if isinstance(checkpoint, dict):
        print(f"🔑 Checkpoint словарь, ключи: {list(checkpoint.keys())}")

        model = Wav2Lip()

        if "state_dict" in checkpoint:
            model.load_state_dict(checkpoint["state_dict"])
            print("✅ Загружен state_dict")
        elif "model" in checkpoint:
            model.load_state_dict(checkpoint["model"])
            print("✅ Загружен model")
        else:
            model.load_state_dict(checkpoint)
            print("✅ Загружен прямой checkpoint")

        model = model.to(device)
        model.eval()
        print("✅ Модель в режиме eval")

    else:
        print("⚠️  TorchScript модель")
        model = torch.jit.load(MODEL_PATH, map_location=device)
        model.eval()
        print("✅ TorchScript модель загружена")

    print("\n✅✅✅ МОДЕЛЬ ГОТОВА К РАБОТЕ!\n")

except Exception as e:
    print(f"❌ ОШИБКА при загрузке модели: {e}")
    print("\nПроверьте что файл Wav2Lip-SD-GAN.pt корректный\n")
    raise

# ==============================================================================
# ШАГ 8: НАСТРОЙКА NGROK
# ==============================================================================
print("="*70)
print("ШАГ 8: НАСТРОЙКА NGROK")
print("="*70)

from pyngrok import ngrok
import getpass

print("🔐 Получите токен на: https://dashboard.ngrok.com/get-started/your-authtoken")
NGROK_TOKEN = getpass.getpass("Введи свой ngrok токен: ")

ngrok.set_auth_token(NGROK_TOKEN)
print("✅ ngrok настроен\n")

# ==============================================================================
# ШАГ 9: СОЗДАНИЕ FLASK СЕРВЕРА
# ==============================================================================
print("="*70)
print("ШАГ 9: СОЗДАНИЕ FLASK СЕРВЕРА")
print("="*70)

from flask import Flask, request, send_file

app = Flask(__name__)

@app.route('/')
def home():
    return "✅ Wav2Lip Inference Server is running!"

@app.route('/run-wav2lip', methods=['POST'])
def run_wav2lip():
    """Основной endpoint для обработки"""
    try:
        if 'image' not in request.files or 'audio' not in request.files:
            return {"error": "Missing image or audio"}, 400

        image_file = request.files['image']
        audio_file = request.files['audio']

        with tempfile.TemporaryDirectory() as tmpdir:
            image_path = os.path.join(tmpdir, 'input_image.jpg')
            audio_path = os.path.join(tmpdir, 'input_audio.wav')
            output_path = os.path.join(tmpdir, 'output.mp4')

            image_file.save(image_path)
            audio_file.save(audio_path)

            print(f"📁 Image: {image_path}")
            print(f"📁 Audio: {audio_path}")

            cmd = [
                'python', 'inference.py',
                '--checkpoint_path', MODEL_PATH,
                '--face', image_path,
                '--audio', audio_path,
                '--outfile', output_path,
                '--nosmooth',
                '--pads', '0', '20', '0', '0'
            ]

            print(f"🚀 Запуск: {' '.join(cmd)}")
            result = subprocess.run(cmd, capture_output=True, text=True)

            if result.returncode != 0:
                print(f"❌ Ошибка: {result.stderr}")
                return {"error": result.stderr}, 500

            if not os.path.exists(output_path):
                return {"error": "Output video not created"}, 500

            print(f"✅ Видео создано: {output_path}")

            return send_file(
                output_path,
                mimetype='video/mp4',
                as_attachment=True,
                download_name='result.mp4'
            )

    except Exception as e:
        print(f"❌ Ошибка: {str(e)}")
        return {"error": str(e)}, 500

@app.route('/health')
def health():
    return {
        "status": "ok",
        "device": device,
        "model": "Wav2Lip SD-GAN"
    }

print("✅ Flask сервер создан\n")

# ==============================================================================
# ШАГ 10: ЗАПУСК СЕРВЕРА
# ==============================================================================
print("="*70)
print("ШАГ 10: ЗАПУСК СЕРВЕРА")
print("="*70)

def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

flask_thread = Thread(target=run_flask, daemon=True)
flask_thread.start()

print("⏳ Ожидание запуска Flask...")
time.sleep(3)

public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*70)
print("✅✅✅ СЕРВЕР ЗАПУЩЕН УСПЕШНО!")
print("="*70)
print(f"\n🌐 PUBLIC URL: {public_url}")
print("\n📋 СКОПИРУЙ И ИСПОЛЬЗУЙ ЭТОТ URL:")
print(f"\n   В файле backend/app/colab_client.py замени:")
print(f"   COLAB_URL = '{public_url}/run-wav2lip'")
print("\n" + "="*70)
print("\n⚠️  ВАЖНО:")
print("   1. НЕ ЗАКРЫВАЙ этот notebook!")
print("   2. Сервер работает пока notebook активен")
print("   3. URL изменится при перезапуске")
print("\n" + "="*70)

# Держим сервер активным
print("\n🔄 Сервер работает... (нажми Ctrl+C для остановки)\n")
try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("\n🛑 Сервер остановлен")

ШАГ 1: ПОДКЛЮЧЕНИЕ GOOGLE DRIVE
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive подключён!

ШАГ 2: УСТАНОВКА ЗАВИСИМОСТЕЙ
✅ Зависимости установлены

ШАГ 3: КЛОНИРОВАНИЕ WAV2LIP
✅ Wav2Lip клонирован
📁 Текущая директория: /content/Wav2Lip

ШАГ 4: УСТАНОВКА REQUIREMENTS
✅ Requirements установлены

ШАГ 5: КОПИРОВАНИЕ МОДЕЛИ
📦 Копируем модель из: /content/drive/MyDrive/Wav2Lip-SD-GAN.pt
✅ Модель скопирована! Размер: 139.07 MB
📁 Путь к модели: checkpoints/Wav2Lip-SD-GAN.pt

ШАГ 6: СКАЧИВАНИЕ FACE DETECTION МОДЕЛИ
📥 Скачиваем face detection модель...
✅ Face detection модель скачана

ШАГ 7: ЗАГРУЗКА МОДЕЛИ WAV2LIP
🖥️  Устройство: cuda
📦 Размер модели: 139.07 MB
🔄 Загружаем модель...

✅ Файл загружен! Тип: <class 'torch.jit._script.RecursiveScriptModule'>
⚠️  TorchScript модель
✅ TorchScript модель загружена

✅✅✅ МОДЕЛЬ ГОТОВА К РАБОТЕ!

ШАГ 8: НАСТРОЙКА NGROK
🔐 Получите токен на: https://dashboard.n

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



✅✅✅ СЕРВЕР ЗАПУЩЕН УСПЕШНО!

🌐 PUBLIC URL: NgrokTunnel: "https://excusively-fluxionary-eusebia.ngrok-free.dev" -> "http://localhost:5000"

📋 СКОПИРУЙ И ИСПОЛЬЗУЙ ЭТОТ URL:

   В файле backend/app/colab_client.py замени:
   COLAB_URL = 'NgrokTunnel: "https://excusively-fluxionary-eusebia.ngrok-free.dev" -> "http://localhost:5000"/run-wav2lip'


⚠️  ВАЖНО:
   1. НЕ ЗАКРЫВАЙ этот notebook!
   2. Сервер работает пока notebook активен
   3. URL изменится при перезапуске


🔄 Сервер работает... (нажми Ctrl+C для остановки)



INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 23:33:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 23:33:38] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 23:35:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Nov/2025 23:54:37] "GET / HTTP/1.1" 200 -
